<a href="https://colab.research.google.com/github/karinayanh/Phyton_Assignments/blob/main/Red_Tomato_Tools.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Red Tomato Tools (RTT)

## Problem Background

Red Tomato Tools (RTT) a small manufacturer of gardening equipment.  
**RTT's production capacity is determined mainly by the size of its workforce** due to minimal equipment and space requirements.

Demand for gardening tools is **highly seasonal**, peaking in the spring.  
To handle seasonality, RTT may:

- hire workers during peak months,  
- lay off workers when demand decreases,  
- build up inventory during low-demand months.

### Key Operational Details

| Parameter | Value |
|----------|-------|
| Selling price | **\$40 / unit** |
| Material cost | **\$10 / unit** |
| Starting workforce (January) | **80 workers** |
| Starting inventory (January) | **1,000 units** |
| Ending inventory requirement (June) | **≥ 500 units** |
| Working days per month | **20 days** |
| Hours per day | **8 hours** |
| Wage rate | **\$4 / hour** |
| Hiring cost | **\$300 / worker** |
| Layoff cost | **\$500 / worker** |
| Inventory holding cost | **\$2 / unit / month** |


### Monthly Demand Forecast

| Month | Demand |
|-------|--------|
| Jan | 1,600 |
| Feb | 3,000 |
| Mar | 3,200 |
| Apr | 3,800 |
| May | 2,200 |
| Jun | 2,200 |

RTT's objective is to find the production schedule that meets demand on time **without stockouts** and minimizes total production and workforce-related costs.

---

## Model Formulation

### Modeling Assumptions

To simplify accounting:

1. All hiring and firing occur at the beginning of the month before production.
2. All demand occurs after production.
3. The monthly storage constraint and the holding cost are based on ending inventory.
4. Each worker provides $20 \text{ days} \times 8 \text{ hours/day} = 160 \text{ hours}$
5. Each worker can produce: $ 160/4 = 40 \text{ units per month}$

---

### Decision Variables

Let:

- $ H_t $: number of workers **hired** in month \(t\)  
- $ L_t $: number of workers **laid off** in month \(t\)  
- $ W_t $: workforce size in month \(t\)  
- $ P_t $: production quantity in month \(t\)  
- $ I_t $: ending inventory in month \(t\)

---

### Objective Function


Minimize total cost over the six months:

$$
\min \sum_{t=1}^{6}
\left(
300H_t + 500L_t + 4*8*20W_t + 10P_t + 2I_t
\right)
$$

Where:

- 300 = hiring cost per worker  
- 500 = layoff cost per worker  
- 4*8*20 = labor cost per worker per month  
- 10 = material cost per unit  
- 2 = holding cost per unit per month  

---

### Constraints

① Workforce balance

Workforce depends on the ending workforce of the previous month and the hiring and firing decision:

$$
W_t = W_{t-1} + H_t - L_t
$$

② Production capacity  

Workers operate at full utilization:

$$
P_t = 20*2W_t
$$

③ Inventory balance

Ending inventory equals beginning inventory plus production minus demand:

$$
I_t = I_{t-1} + P_t - D_t
$$

④ Demand fulfillment  
Inventory on hand must be sufficient:

$$
I_{t-1} + P_t \ge D_t
$$

⑤ Starting values  
$$
W_0 = 80, \quad I_0 = 1000
$$

⑥ Ending inventory requirement  
$$
I_6 \ge 500
$$

⑦ Non-negativity  
$$
H_t, L_t, W_t, P_t, I_t \ge 0
$$

---

## Python Implementation (Using Pyomo + HiGHS)


Below are the key functions we use and what they do:

- **`pyo.ConcreteModel()`**  

- **`pyo.Set(initialize=...)`**: <span style="color:red">Defines an index set, which is used to represent the planning months $t = 1,\dots,6$.</span>

- **`pyo.Var(model.T, domain=pyo.NonNegativeReals)`**: <span style="color:red">Defines decision variables indexed by the month index set. </span>

- **`pyo.Objective(expr=..., sense=pyo.minimize)`**  

- **`pyo.Constraint(model.T, rule=...)`**: <span style="color:red">Each constraint is indexed by month and enforced using a `rule`.</span>

- **`pyo.SolverFactory("appsi_highs")`**  

- **`solver.solve(model)`**  

- **`pyo.value(model.variable[t])`**  

In [ ]:
import pyomo.environ as pyo
from openpyxl import load_workbook
import re

# 0. Read ALL data & parameters from Excel
file_path = "/content/Red Tomato Tools.xlsx"
wb = load_workbook(file_path, data_only=True)
ws = wb["Sheet1"]

# Initial conditions (Period 0)
W0 = float(ws["D4"].value)   # starting workforce
I0 = float(ws["E4"].value)   # starting inventory

# Planning horizon
months = [1, 2, 3, 4, 5, 6]

# Demand forecast for the six-month period  D_t is in G5:G10
D = {t: float(ws[f"G{t+4}"].value) for t in months}

# Operating assumptions
days_per_month = float(ws["J3"].value)    # Operating days per month
hours_per_day  = float(ws["J4"].value)    # Operating hours per day

# Cost / parameter table from Excel (J5:J10)
holding_cost  = float(ws["J5"].value)   # $/unit/month
hire_cost     = float(ws["J6"].value)   # $/worker hired
layoff_cost   = float(ws["J7"].value)   # $/worker laid off
labour_hours_required = float(ws["J8"].value)  # hours/unit
material_cost = float(ws["J9"].value)   # $/unit
wage_per_hour = float(ws["J10"].value)  # $/hour

# Ending inventory requirement
ending_text = str(ws["E10"].value) # Excel cell E10 contains text like "at least 500"
I6_min = float(re.findall(r"\d+\.?\d*", ending_text)[0])  # extract 500

# Derived parameters (same logic as slides)
hours_per_worker = days_per_month * hours_per_day     # 20*8 = 160 hours/worker-month
units_per_worker = hours_per_worker / labour_hours_required  # 160/4 = 40 units/worker-month
labor_cost_per_worker = wage_per_hour * hours_per_worker     # 4*160 = 640 $/worker-month


# 1. Build the model (continuous workforce as in slides)
model = pyo.ConcreteModel()
model.T = pyo.Set(initialize=months)

# Decision variables (continuous aggregate planning)
model.H = pyo.Var(model.T, domain=pyo.NonNegativeReals)  # hires in month t
model.L = pyo.Var(model.T, domain=pyo.NonNegativeReals)  # layoffs in month t
model.W = pyo.Var(model.T, domain=pyo.NonNegativeReals)  # workforce in month t
model.P = pyo.Var(model.T, domain=pyo.NonNegativeReals)  # production in month t
model.I = pyo.Var(model.T, domain=pyo.NonNegativeReals)  # ending inventory in month t

# Objective: minimize total cost over 6 months
model.obj = pyo.Objective(
    expr=sum(
        hire_cost * model.H[t]
        + layoff_cost * model.L[t]
        + labor_cost_per_worker * model.W[t]
        + material_cost * model.P[t]
        + holding_cost * model.I[t]
        for t in model.T
    ),
    sense=pyo.minimize
)


# 2. Constraints
# a. Workforce balance: W_t = W_{t-1} + H_t - L_t
model.workforce_balance = pyo.Constraint(
    model.T,
    rule=lambda m, t: (
        m.W[t] == W0 + m.H[t] - m.L[t] if t == 1
        else m.W[t] == m.W[t-1] + m.H[t] - m.L[t]
    )
)


# b. Full-utilization production capacity: P_t = units_per_worker * W_t
model.capacity = pyo.Constraint( model.T, rule=lambda m, t: m.P[t] == units_per_worker * m.W[t])

# c. Inventory balance: I_t = I_{t-1} + P_t - D_t
model.inventory_balance = pyo.Constraint(
    model.T,
    rule=lambda m, t: (
        m.I[t] == I0 + m.P[t] - D[t] if t == 1
        else m.I[t] == m.I[t-1] + m.P[t] - D[t]
    )
)

# d. Ending inventory requirement:
model.ending_inventory = pyo.Constraint(expr=model.I[6] >= I6_min)


# 3. Solve
solver = pyo.SolverFactory("appsi_highs")
solver.solve(model)

print("Total cost = $", round(pyo.value(model.obj), 2)) # The optimal total cost of the 6-month aggregate plan is $ 423675.

print("\nMonth | Hired | Laid off | Workforce | Production | Demand | Ending Inv")
print("-"*75)
for t in months:
    print(f"{t:>5} |"
          f"{pyo.value(model.H[t]):>6.2f} |" # The numbers of workers hired in months 1-6 are: 0.00, 0.00, 0.00, 0.00, 0.00, 0.00
          f"{pyo.value(model.L[t]):>9.2f} |" # The numbers of workers laid off in months 1-6 are: 13.75, 0.00, 0.00, 0.00, 5.00, 0.00
          f"{pyo.value(model.W[t]):>9.2f} |" # Workforce levels in months 1-6 are: 66.25, 66.25, 66.25, 66.25, 61.25, 61.25
          f"{pyo.value(model.P[t]):>10.0f} |" # Production quantities in months 1-6 are: 2650, 2650, 2650, 2650, 2450, 2450
          f"{D[t]:>6.0f} |" # Monthly demand values: 1600, 3000, 3200, 3800, 2200, 2200
          f"{pyo.value(model.I[t]):>10.0f}") # Ending inventory each month: 2050, 1700, 1150, 0, 250, 500


Total cost = $ 423675.0

Month | Hired | Laid off | Workforce | Production | Demand | Ending Inv
---------------------------------------------------------------------------
    1 |  0.00 |    13.75 |    66.25 |      2650 |  1600 |      2050
    2 |  0.00 |     0.00 |    66.25 |      2650 |  3000 |      1700
    3 |  0.00 |     0.00 |    66.25 |      2650 |  3200 |      1150
    4 |  0.00 |     0.00 |    66.25 |      2650 |  3800 |         0
    5 |  0.00 |     5.00 |    61.25 |      2450 |  2200 |       250
    6 |  0.00 |     0.00 |    61.25 |      2450 |  2200 |       500
